# Mount to google drive


In [1]:
!pip install open_clip_torch
from google.colab import drive
drive.mount('/content/drive')


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.8 MB/s eta 0:00:00
Mounted at /content/drive


In [2]:
from pathlib import Path

DATA_ROOT = Path("/content/drive/MyDrive/VLM_project/HatefulMemes/data")
PROJECT_ROOT = Path("/content/drive/MyDrive/VLM_project")

# IMPORTANT: root data dir, not img/
IMAGE_DIR = DATA_ROOT

TRAIN_JSON = DATA_ROOT / "train_clean.jsonl"

LOSF_EVAL = PROJECT_ROOT / "losf_eval.py"
METRICS_DIR = PROJECT_ROOT / "metrics"
METRICS_DIR.mkdir(parents=True, exist_ok=True)

print("DATA_ROOT:", DATA_ROOT)
print("IMAGE_DIR:", IMAGE_DIR)
print("TRAIN_JSON exists:", TRAIN_JSON.exists())
print("LOSF_EVAL exists:", LOSF_EVAL.exists())
print("METRICS_DIR:", METRICS_DIR)


DATA_ROOT: /content/drive/MyDrive/VLM_project/HatefulMemes/data
IMAGE_DIR: /content/drive/MyDrive/VLM_project/HatefulMemes/data
TRAIN_JSON exists: True
LOSF_EVAL exists: True
METRICS_DIR: /content/drive/MyDrive/VLM_project/metrics


In [3]:
import torch, sys

print("Python:", sys.version)
print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

# open_clip import test
import open_clip
print("open_clip:", open_clip.__version__ if hasattr(open_clip, "__version__") else "import ok")


Python: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
Torch: 2.9.0+cpu
CUDA available: False
open_clip: 3.2.0


Cell 2 — Run LOSF eval (debug first)

In [4]:
import subprocess, json

OUT_LOSF_DEBUG = METRICS_DIR / "losf_metrics_train_debug_k5_beta1_limit300.json"

cmd = [
    "python", str(LOSF_EVAL),
    "--image_dir", str(IMAGE_DIR),
    "--json_path", str(TRAIN_JSON),
    "--output_json", str(OUT_LOSF_DEBUG),
    "--k", "5",
    "--device", "cuda",
    "--limit", "300",
    "--beta", "1.0",
]

print("Running:\n", " ".join(cmd))
res = subprocess.run(cmd, capture_output=True, text=True)

print("STDOUT:\n", res.stdout[:4000])
print("STDERR:\n", res.stderr[:4000])
print("Return code:", res.returncode)

assert OUT_LOSF_DEBUG.exists(), f"Missing output json: {OUT_LOSF_DEBUG}"
print("Wrote:", OUT_LOSF_DEBUG)

with open(OUT_LOSF_DEBUG, "r", encoding="utf-8") as f:
    losf_debug = json.load(f)
losf_debug


Running:
 python /content/drive/MyDrive/VLM_project/losf_eval.py --image_dir /content/drive/MyDrive/VLM_project/HatefulMemes/data --json_path /content/drive/MyDrive/VLM_project/HatefulMemes/data/train_clean.jsonl --output_json /content/drive/MyDrive/VLM_project/metrics/losf_metrics_train_debug_k5_beta1_limit300.json --k 5 --device cuda --limit 300 --beta 1.0
STDOUT:
 
STDERR:
 /content/drive/MyDrive/VLM_project/losf_eval.py:86: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=use_amp):
/content/drive/MyDrive/VLM_project/losf_eval.py:105: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=use_amp):

Return code: 0
Wrote: /content/drive/MyDrive/VLM_project/metrics/losf_metrics_train_debug_k5_beta1_limit300.json


{'utility_recall@k_pre_S_V': 0.8552631578947368,
 'utility_recall@k_post_S_V': 0.8333333333333334,
 'harmful_recall@k_pre_U_V': 0.8888888888888888,
 'harmful_recall@k_post_U_V': 0.8333333333333334,
 'text_semantic_shift_decline_to_unsafe': 0.0,
 'text_semantic_shift_increase_to_neutral': 0.0,
 'clipscore_safe_pre': 0.3332764804363251,
 'clipscore_safe_post': 0.3333534896373749,
 'safety_rates_ASR': None,
 'safety_rates_USR': None}

Cell 3 — Run LOSF eval (full train)

In [ ]:
OUT_LOSF_TRAIN = METRICS_DIR / "losf_metrics_train_k5_beta1.json"

cmd = [
    "python", str(LOSF_EVAL),
    "--image_dir", str(IMAGE_DIR),
    "--json_path", str(TRAIN_JSON),
    "--output_json", str(OUT_LOSF_TRAIN),
    "--k", "5",
    "--device", "cuda",
    # "--limit", "5000",
    "--beta", "1.0",
    "--batch_size", "32",
]

print("Running:\n", " ".join(cmd))
res = subprocess.run(cmd, capture_output=True, text=True)

print("STDOUT:\n", res.stdout[:4000])
print("STDERR:\n", res.stderr[:4000])
print("Return code:", res.returncode)

assert OUT_LOSF_TRAIN.exists(), f"Missing output json: {OUT_LOSF_TRAIN}"
print("Wrote:", OUT_LOSF_TRAIN)

with open(OUT_LOSF_TRAIN, "r", encoding="utf-8") as f:
    losf_train = json.load(f)

losf_train


Running:
 python /content/drive/MyDrive/VLM_project/losf_eval.py --image_dir /content/drive/MyDrive/VLM_project/HatefulMemes/data --json_path /content/drive/MyDrive/VLM_project/HatefulMemes/data/train_clean.jsonl --output_json /content/drive/MyDrive/VLM_project/metrics/losf_metrics_train_k5_beta1.json --k 5 --device cuda --beta 1.0 --batch_size 32


# plot sublimit300

In [ ]:
from pathlib import Path
import json

losf_train_out = Path(
    "/content/drive/MyDrive/VLM_project/metrics/losf_metrics_train_debug_k5_beta1_limit300.json"
)

print("File exists:", losf_train_out.exists())

with open(losf_train_out, "r") as f:
    losf_metrics_train_debug = json.load(f)

losf_metrics_train_debug


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

losf_df = pd.Series(losf_metrics_train_debug)
losf_df["method"] = "LOSF"
losf_df = losf_df.to_frame().T


In [ ]:
def plot_pre_post(df, metric_pre, metric_post, title):
    fig, ax = plt.subplots(figsize=(4,3))

    methods = df["method"].tolist()
    pre_vals = df[metric_pre].astype(float).tolist()
    post_vals = df[metric_post].astype(float).tolist()

    x = np.arange(len(methods))
    width = 0.35

    ax.bar(x - width/2, pre_vals, width, label="pre")
    ax.bar(x + width/2, post_vals, width, label="post")

    ax.set_xticks(x)
    ax.set_xticklabels(methods)
    ax.set_ylabel("value")
    ax.set_title(title)
    ax.legend()
    plt.tight_layout()
    plt.show()


In [ ]:
plot_pre_post(
    losf_df,
    "utility_recall@k_pre_S_V",
    "utility_recall@k_post_S_V",
    "LOSF: Utility Recall@K (safe, train_debug)"
)


plot_pre_post(
    losf_df,
    "harmful_recall@k_pre_U_V",
    "harmful_recall@k_post_U_V",
    "LOSF: Harmful Recall@K (unsafe, train_debug)"
)

plot_pre_post(
    losf_df,
    "clipscore_safe_pre",
    "clipscore_safe_post",
    "LOSF: CLIPScore Safe Pairs (train_debug)"
)
